In [1]:
import numpy as np
import pickle
from tqdm.notebook import trange, tqdm

In [11]:
'''
This script is able to build a coocurence matrix from a list of COG of an organism.
It needs to take a csv file containing the list of the COG and the directory where the
COG files are stored.
It returns a numpy array.
'''
def iter_query_cog(query_orth):
    with open(query_orth) as csvfile:
        next(csvfile)
        for l in csvfile:
            buffer = l.split(';')
            yield([buffer[4],buffer[5]])

def cog_2_g_list(cog_id,cog_dir):
    file_path = f'{cog_dir}/COG_{cog_id}.csv'
    g_list = list()
    with open(file_path) as csvfile:
        next(csvfile)
        for l in csvfile:
            buffer = l.split(',')
            strain = buffer[1]
            g_list.append(strain)
    return g_list

def p_dict_builder(query_orth,cog_dir):
    p_dict={}
    i=0
    for cog_id,p_name in iter_query_cog(query_orth):
        g_list = cog_2_g_list(cog_id,cog_dir)
        p_dict[p_name] = g_list
        #i=i+1
        #if i>9:
            #break
    return p_dict

def index_setter(p_dict):
    p_index = list()
    g_index = set()
    for p_name,g_list in p_dict.items():
        p_index.append(p_name)
        g_index.update(g_list)
    g_index = list(g_index)
    return p_index,g_index

def set_M_values(M,m_index,g_name,p_name,value): 
    i = m_index[0].index(p_name) #protein index
    j = m_index[1].index(g_name) #genome index
    M[i,j] = value

def matrix_builder(query_orth,cog_dir,value):
    #generate a dictionnary of protein coocurences
    p_dict=p_dict_builder(query_orth,cog_dir)
    with open('p_dict.pkl', 'wb') as file:
        pickle.dump(p_dict, file)
    #generate the coocurences matrix index
    m_index=index_setter(p_dict)
    with open('m_index.pkl', 'wb') as file:
        pickle.dump(m_index, file)
    #generate the coocurences matrix
    x = len(m_index[0])
    y = len(m_index[1])
    M = np.zeros((x,y))
    for p_name,g_list in p_dict.items():
        for g_name in g_list:
            set_M_values(M,m_index,g_name,p_name,value)
    with open('p_matrix.npy', 'wb') as f:
        np.save(f, M)
    return M

In [4]:
def score_matrix_builder(score_file_dir):
    path = score_file_dir
    dico_prot={}
    for file in os.listdir(path):
        dico_strain={}
        protein = file.strip('_scores.txt')
        file_path = f'{path}{file}'
        with open(file_path,'r') as file:
            for l in file:
                buffer = l.split(',')
                strain = buffer[2]
                score = buffer[3].strip('\n')
                dico_strain[strain]=score
        dico_prot[protein]=dico_strain
    return(dico_prot)        

In [ ]:
%%time
query_orth ='/Users/mdupuy/Documents/Stage/Pseudomonas_aeruginosa_PA7_119_ortholog_groups.csv'
cog_dir='/Users/mdupuy/Documents/Stage/All_COG_groups/'
matrix_builder(query_orth,cog_dir,1)

In [ ]:
%%time
p_dict=p_dict_builder(query_orth,cog_dir)
m_index=index_setter(p_dict)

In [ ]:
%%time
with open('p_dict.pkl','rb') as file:
    p_dict = pickle.load(file)
with open('m_index.pkl','rb') as file:
    m_index = pickle.load(file)
x = len(m_index[0])
y = len(m_index[1])
M = np.zeros((x,y))
for p_name,g_list in p_dict.items():
    for g_name in g_list:
        set_M_values(M,m_index,g_name,p_name,1)

In [14]:
%%time
import os
score_file_dir = '/Users/mdupuy/Documents/Stage/Parser/Scores/'
dico_score = score_matrix_builder(score_file_dir)
m_index = index_setter(dico_score)

(['PSPA7_3376', 'PSPA7_2475', 'PSPA7_3669', 'PSPA7_6369', 'PSPA7_1922', 'PSPA7_2608', 'PSPA7_3414', 'PSPA7_6114', 'PSPA7_0943', 'PSPA7_2317', 'PSPA7_2320', 'PSPA7_3423', 'PSPA7_6123', 'PSPA7_0974', 'PSPA7_2442', 'PSPA7_5287', 'PSPA7_0787', 'PSPA7_1915', 'PSPA7_3341', 'PSPA7_5798', 'PSPA7_4584', 'PSPA7_0298', 'PSPA7_1084', 'PSPA7_2203', 'PSPA7_5848', 'PSPA7_6000', 'PSPA7_0857', 'PSPA7_3500', 'PSPA7_1836', 'PSPA7_2561', 'PSPA7_3262', 'PSPA7_4829', 'PSPA7_1190', 'PSPA7_3255', 'PSPA7_2049', 'PSPA7_4490', 'PSPA7_1801', 'PSPA7_0693', 'PSPA7_2556', 'PSPA7_5393', 'PSPA7_6037', 'PSPA7_0860', 'PSPA7_3537', 'PSPA7_3028', 'PSPA7_2234', 'PSPA7_2375', 'PSPA7_3169', 'PSPA7_3476', 'PSPA7_0921', 'PSPA7_6176', 'PSPA7_2417', 'PSPA7_2899', 'PSPA7_1940', 'PSPA7_2108', 'PSPA7_3314', 'PSPA7_4968', 'PSPA7_3323', 'PSPA7_2420', 'PSPA7_1977', 'PSPA7_5098', 'PSPA7_4284', 'PSPA7_3441', 'PSPA7_0598', 'PSPA7_0916', 'PSPA7_1784', 'PSPA7_6141', 'PSPA7_5909', 'PSPA7_5587', 'PSPA7_2342', 'PSPA7_0087', 'PSPA7_3200', 'PSP

In [ ]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
def hamming(coocurencies_matrix):
    hamming = pdist(coocurencies_matrix, metric='hamming')
    distance_matrix = squareform(hamming)
    with open('hd_matrix.pkl', 'wb') as f:
        pickle.dump(distance_matrix,f)
    return distance_matrix

In [ ]:
d_M = hamming(M)